<a href="https://colab.research.google.com/github/guptapawan227/Capstone_AIML/blob/Pawan/31_Dec_Visualiztion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip3 install ftfy

     |████████████████████████████████| 71kB 8.2MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=b765be24ee1364c73319bee5990f623350224d712f42d20156f64e6bae786ed8
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy


Importing Libraries

In [ ]:
# Using TensorFlow 1.x only in colab as found a issue with 2.3 version used by colab while working with DNN model fit. Did not observe any issue with Tensor flow 2.1 version on local jupyter enviornment.
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import time, os, sys, itertools, re 
from PIL import Image
import warnings, pickle, string
from dateutil import parser
%matplotlib inline

# Data Visualization
import cufflinks as cf
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

from ftfy import fix_text, badness

# Traditional Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Sequential Modeling
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers.merge import Concatenate
from keras.layers import Input, Dropout, Flatten, Dense, Embedding, LSTM, GRU
from keras.layers import BatchNormalization, TimeDistributed, Conv1D, MaxPooling1D
from keras.constraints import max_norm, unit_norm
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint


# Tools & Evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report, auc
from sklearn.metrics import roc_curve, accuracy_score, precision_recall_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Reading the data from excel 

## Language Translation

#### Load the consolidated final translated pickle file which contains the language translations. The Process used for language translation is commented below

In [ ]:
!pip install langdetect

In [ ]:
with open('/content/drive/MyDrive/Capstone/Final_Translated_combined.pkl','rb') as f:
#with open('Final_Translated_combined.pkl','rb') as f:
    clean_data = pickle.load(f)

In [ ]:
from langdetect import detect
    
def fn_lang_detect(df):                                        
   try:                                                          
      return detect(df)                                      
   except:                                                       
      return 'no'                                                  

clean_data['language'] = clean_data['Clean_Description'].apply(fn_lang_detect)

In [ ]:
x = clean_data["language"].value_counts()
x=x.sort_index()
plt.figure(figsize=(10,6))
ax= sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Distribution of text by language")
plt.ylabel('number of records')
plt.xlabel('Language')
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show();

NameError: ignored

In [ ]:
data1 = clean_data[(clean_data["language"] != 'zh-cn') & (clean_data["language"] != 'de')]
data1

In [ ]:
data1.isnull().sum()

In [ ]:
# Load the translated pickle file 
#with open('/content/drive/MyDrive/Capstone_AIML_Pawan/clean_data_translated.pkl','rb') as f:
with open('clean_data_translated.pkl','rb') as f:
    clean_data_translated_pkl = pickle.load(f)

FileNotFoundError: ignored

In [ ]:
clean_data_translated_pkl

In [ ]:
clean_data_translated_pkl.isnull().sum()

In [ ]:
clean_data_translated_pkl=clean_data_translated_pkl.drop(['Clean_Description', 'Unnamed: 0'], axis=1)

In [ ]:
clean_data_translated_pkl.rename(columns = {'Translation_Text': 'Clean_Description'}, inplace = True)

In [ ]:
clean_data_translated_pkl

In [ ]:
dataframes=[data1,clean_data_translated_pkl]
clean_data_final_2= pd.concat(dataframes)

In [ ]:
clean_data_final_2

In [ ]:
clean_data_final_2.isnull().sum()

In [ ]:
clean_data_final_2 = clean_data_final_2[(clean_data_final_2['Assignment group'] != 'GRP_32')]

In [ ]:
clean_data_final_2.to_csv('clean_data_final_2.csv', index=False, encoding='utf_8_sig')
#with open('/content/drive/MyDrive/Capstone/Final_Translated_combined.pkl','wb') as f:
with open('Final_Translated_combined.pkl','wb') as f:
    pickle.dump(clean_data_final_2, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the consolidated final translated pickle file 
#with open('/content/drive/MyDrive/Capstone/Final_Translated_combined.pkl','rb') as f:
with open('Final_Translated_combined.pkl','rb') as f:
    clean_data = pickle.load(f)

In [ ]:
clean_data.tail()

In [ ]:
assignment_group_cnt=clean_data['Assignment group'].value_counts()
assignment_group_cnt.describe()

In [ ]:
assignment_group_cnt.tail(50)

In [ ]:
clean_data

## Data Augmentation

In [ ]:
#Install NLPAug Package
!pip install nlpaug

In [ ]:
#Install dependencies for nlpaug
!pip install torch>=1.6.0 transformers>=4.0.0
!pip install nltk>=3.4.5

In [ ]:
#We will use Word Augmentation methods. nlpaug supports character, word and sentence level augmentation methods
import nlpaug.augmenter.word as naw

In [ ]:
#Word embedding augmentation method
aug1 = naw.WordEmbsAug(model_type='glove', model_path='glove.6B/glove.6B.50d.txt', action="substitute")


In [ ]:
aug2 = naw.WordEmbsAug(model_type='glove', model_path='glove.6B/glove.6B.50d.txt', action="insert")


In [ ]:
#Contextual Word augmentation method
aug3 = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")


In [ ]:
aug4 = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")


In [ ]:
#Synonym Augmentation method using PPDB models downloaded from http://paraphrase.org/#/download
aug5 = naw.SynonymAug(aug_src='ppdb', model_path='ppdb-2.0-s-all')


In [ ]:
#!pip3 install nltk
import nltk 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

In [ ]:
#Create a new dataframe with records not in GRP_0
zero_dataframe = clean_data[clean_data["Assignment group"] == 'GRP_0']
new_dataframe = clean_data[clean_data["Assignment group"] != 'GRP_0']
zero_dataframe.head()


In [ ]:
#Create dataframe copies for different augmentation methods
new_dataframe2 = new_dataframe.copy()
new_dataframe3 = new_dataframe.copy()
new_dataframe4 = new_dataframe.copy()
new_dataframe5 = new_dataframe.copy()

In [ ]:
new_dataframe.shape, zero_dataframe.shape, new_dataframe2.shape, new_dataframe3.shape, new_dataframe4.shape, new_dataframe5.shape

In [ ]:
maxsyn=2
new_dataframe["Augmented_data"] = new_dataframe.apply(lambda x: create_set_of_new_sentences(x['Clean_Description'], maxsyn),axis=1)
new_dataframe

In [ ]:
new_dataframe["Augmented_data"] = new_dataframe.apply(lambda x: aug1.augment(x['Clean_Description']),axis=1)
new_dataframe

In [ ]:
s = new_dataframe.apply(lambda x: pd.Series(x['Augmented_data']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Final_Text'
new_dataframe_aug = new_dataframe.drop(['New Description','Augmented_data', 'Clean_Description'],axis=1).join(s)
new_dataframe_aug

In [ ]:
new_dataframe2["Augmented_data"] = new_dataframe2.apply(lambda x: aug2.augment(x['Clean_Description']),axis=1)
new_dataframe2

In [ ]:
s = new_dataframe2.apply(lambda x: pd.Series(x['Augmented_data']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Final_Text'
new_dataframe_aug2 = new_dataframe2.drop(['New Description','Augmented_data', 'Clean_Description'],axis=1).join(s)
new_dataframe_aug2

In [ ]:
new_dataframe3["Augmented_data"] = new_dataframe3.apply(lambda x: aug3.augment(x['Clean_Description']),axis=1)
new_dataframe3

In [ ]:
s = new_dataframe3.apply(lambda x: pd.Series(x['Augmented_data']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Final_Text'
new_dataframe_aug3 = new_dataframe3.drop(['New Description','Augmented_data', 'Clean_Description'],axis=1).join(s)
new_dataframe_aug3

In [ ]:
new_dataframe4["Augmented_data"] = new_dataframe4.apply(lambda x: aug4.augment(x['Clean_Description']),axis=1)
new_dataframe4

In [ ]:
s = new_dataframe4.apply(lambda x: pd.Series(x['Augmented_data']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Final_Text'
new_dataframe_aug4 = new_dataframe4.drop(['New Description','Augmented_data', 'Clean_Description'],axis=1).join(s)
new_dataframe_aug4

In [ ]:
new_dataframe5["Augmented_data"] = new_dataframe5.apply(lambda x: aug5.augment(x['Clean_Description']),axis=1)
new_dataframe5

In [ ]:
s = new_dataframe5.apply(lambda x: pd.Series(x['Augmented_data']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Final_Text'
new_dataframe_aug5 = new_dataframe5.drop(['New Description','Augmented_data', 'Clean_Description'],axis=1).join(s)
new_dataframe_aug5

In [ ]:
zero_dataframe = zero_dataframe.rename(columns={"Clean_Description": "Final_Text"})
zero_dataframe = zero_dataframe.drop(['New Description'], axis = 1)


In [ ]:
zero_dataframe

In [ ]:
#Adding Original data (without Augmentation)
new_dataframe6 = clean_data[clean_data["Assignment group"] != 'GRP_0']
new_dataframe6 = new_dataframe6.rename(columns={"Clean_Description": "Final_Text"})
new_dataframe6 = new_dataframe6.drop(['New Description'], axis = 1)
new_dataframe6

In [ ]:
dataframes=[new_dataframe_aug, new_dataframe_aug2, new_dataframe_aug3, new_dataframe_aug4, new_dataframe_aug5, zero_dataframe, new_dataframe6]
clean_data_result= pd.concat(dataframes)
clean_data_result

In [ ]:
#Remove duplicate rows after augmentation
clean_data_result = clean_data_result.drop_duplicates(subset='Final_Text', keep="first")

In [ ]:
# Serialize the Augmented dataset for later use
clean_data_result.to_csv('Interim_data.csv', index=False, encoding='utf_8_sig')
#with open('/content/Interim_data.pkl','wb') as f:
with open('Interim_data.pkl','wb') as f:
    pickle.dump(clean_data_result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the consolidated final translated pickle file 
#with open('/content/drive/MyDrive/Capstone/Final_Translated_combined.pkl','rb') as f:
with open('Interim_data.pkl','rb') as f:
    clean_data_result = pickle.load(f)

In [ ]:
clean_data_result

## Stop words removal and Lemmatise text

In [ ]:
clean_data_result.isnull().sum()

In [ ]:
import re
import string
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english')) 

processed_all_documents = list()

for desc in clean_data_result['Final_Text']:
    word_tokens = word_tokenize(desc) 
    
    filtered_sentence = [] 

    # Removing Stopwords
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    words = ' '.join(filtered_sentence)
    processed_all_documents.append(words)  

In [ ]:
clean_data_result['Final_Text'] = processed_all_documents

In [ ]:
clean_data_result.head(50)

In [ ]:
clean_data_result.dropna()

In [ ]:
clean_data_result.isnull().sum()

In [ ]:
#Lemmatisation using spacy library
!pip install spacy

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [ ]:
!pip3 install spacy

In [ ]:
# Need to run "python -m spacy download en" in anaconda prompt to avoid 'en' not found issue.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

clean_data_result['Final_Text'] = clean_data_result['Final_Text'].apply(lemmatize_text)

In [ ]:
clean_data_result

In [ ]:
clean_data_result1 = clean_data_result[clean_data_result['Final_Text'] != '']

In [ ]:
clean_data_result1

In [ ]:
assignment_group_cnt=clean_data_result1['Assignment group'].value_counts()
assignment_group_cnt.describe()

In [ ]:
assignment_group_cnt.tail(24)

In [ ]:
# Serialize the translated dataset
clean_data_result1.to_csv('Final_data.csv', index=False, encoding='utf_8_sig')
#with open('/content/Final_data.pkl','wb') as f:
with open('Final_data.pkl','wb') as f:
    pickle.dump(clean_data_result1, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the translated pickle file 
#with open('/content/Final_data.pkl','rb') as f:
with open('Final_data.pkl','rb') as f:
    clean_data = pickle.load(f)

## Prepping Dataframe for Model Building

In [ ]:
clean_data

In [ ]:
'''# Import label encoder 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
clean_data['Assignment group LabelEncoded']= label_encoder.fit_transform(clean_data['Assignment group']) 
  


onehot_encoder = OneHotEncoder()
clean_data['Assignment group LabelEncoded'] = clean_data['Assignment group LabelEncoded'].values.reshape(len(clean_data['Assignment group LabelEncoded']), 1)
clean_data['Assignment group OneHotEncoded'] = onehot_encoder.fit_transform(clean_data[['Assignment group LabelEncoded']])
clean_data['Assignment group OneHotEncoded']'''

In [ ]:
# Import label encoder 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
clean_data['Assignment group LabelEncoded']= label_encoder.fit_transform(clean_data['Assignment group']) 

target_strings = label_encoder.inverse_transform(clean_data['Assignment group LabelEncoded'])
target_strings_list = np.unique(target_strings).tolist()
clean_data['Assignment group LabelEncoded']

In [ ]:
'''# Create a target categorical column
clean_data['Assignment group LabelEncoded'] = clean_data['Assignment group'].astype('category').cat.codes
clean_data.info()'''

In [ ]:
clean_data

In [ ]:
clean_data.isnull().sum()

In [ ]:
label_encoded_dict = dict(zip(clean_data['Assignment group'].unique(), clean_data['Assignment group LabelEncoded'].unique()))
label_encoded_dict

In [ ]:
# Splitting Train Test 
from sklearn.model_selection import train_test_split
#Y = np.array(clean_data['Assignment group OneHotEncoded'])

X_train, X_test, y_train1, y_test1 = train_test_split(clean_data['Final_Text'], clean_data['Assignment group LabelEncoded'], test_size=0.3, random_state = 0, stratify=clean_data['Assignment group LabelEncoded'])
print('\033[1mShape of the training set:\033[0m', X_train.shape, X_test.shape)
print('\033[1mShape of the test set:\033[0m', y_train.shape, y_test.shape)

In [ ]:
#Using TFIDF vectors feature extraction
Tfidf_vect = TfidfVectorizer(max_features=2500)
Tfidf_vect.fit(clean_data.Final_Text.astype(str))
X_train_tfidf = Tfidf_vect.transform(X_train)
X_test_tfidf = Tfidf_vect.transform(X_test)

In [ ]:
#using count vectorizer for features extraction
from sklearn.feature_extraction.text import CountVectorizer

CV = CountVectorizer(max_features=2500)

X_BoW = CV.fit_transform(clean_data['Final_Text']).toarray()
y = clean_data['Assignment group LabelEncoded']

print("Shape of Input Feature :",np.shape(X_BoW))
print("Shape of Target Feature :",np.shape(y))

In [ ]:
# Splitting Train Test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_BoW, y, test_size=0.3, random_state = 0, stratify=y)
print('\033[1mShape of the training set:\033[0m', X_train.shape, X_test.shape)
print('\033[1mShape of the test set:\033[0m', y_train.shape, y_test.shape)

In [ ]:
def run_classification(estimator, X_train, X_test, y_train, y_test, arch_name=None, pipelineRequired=True, isDeepModel=False):
    # train the model
    clf = estimator

    if pipelineRequired :
        clf = Pipeline([('tfidf', TfidfTransformer()),
                     ('clf', estimator),
                     ])
      
    if isDeepModel :
        clf.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=25, batch_size=128,verbose=1,callbacks=call_backs(arch_name))
        # predict from the clasiffier
        y_pred = clf.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        y_train_pred = clf.predict(X_train)
        y_train_pred = np.argmax(y_train_pred, axis=1)
    else :
        clf.fit(X_train, y_train)
        # predict from the clasiffier
        y_pred = clf.predict(X_test)
        y_train_pred = clf.predict(X_train)
    
    print('Estimator:', clf)
    print('='*80)
    print('Training accuracy: %.2f%%' % (accuracy_score(y_train,y_train_pred) * 100))
    print('Testing accuracy: %.2f%%' % (accuracy_score(y_test, y_pred) * 100))
    print('='*80)
    print('Confusion matrix:\n %s' % (confusion_matrix(y_test, y_pred)))
    print('='*80)
    print('Classification report:\n %s' % (classification_report(y_test, y_pred, target_names=target_strings_list)))
    

## Logistic Regression

In [ ]:
run_classification(LogisticRegression(C=5, penalty='l2', solver='liblinear'), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
run_classification(LogisticRegression(C=5, penalty='l2', solver='liblinear'), X_train, X_test, y_train, y_test)

## Naive Bayes Classifier

In [ ]:
run_classification(MultinomialNB(alpha=0, class_prior=None, fit_prior='True'), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
run_classification(MultinomialNB(alpha=0, class_prior=None, fit_prior='True'), X_train, X_test, y_train, y_test)

## K-nearest Neighbor

In [ ]:
run_classification(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance'), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
run_classification(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance'), X_train, X_test, y_train, y_test)

## Support Vector Machine (SVM)

In [ ]:
run_classification(LinearSVC(C=1), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
run_classification(LinearSVC(C=1), X_train, X_test, y_train, y_test)

## Decision Tree

In [ ]:
run_classification(DecisionTreeClassifier(criterion='gini', min_samples_leaf=2), X_train, X_test, y_train, y_test)

In [ ]:
run_classification(DecisionTreeClassifier(criterion='gini', min_samples_leaf=2), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

## Random Forest

In [ ]:
run_classification(RandomForestClassifier(criterion= 'entropy', n_estimators=100, random_state=0), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
run_classification(RandomForestClassifier(criterion= 'entropy', n_estimators=100, random_state=0), X_train, X_test, y_train, y_test)

## GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
run_classification(GradientBoostingClassifier(max_depth=15, n_estimators=50, random_state=42), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
run_classification(GradientBoostingClassifier(max_depth=15, n_estimators=50, random_state=42), X_train, X_test, y_train, y_test)

## XGBoosting

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
run_classification(XGBClassifier(n_estimators=50, max_depth=15), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

## Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
run_classification(BaggingClassifier(n_estimators=100, random_state=10), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

## Stacking

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)), ('svr', make_pipeline(StandardScaler(with_mean=False), LinearSVC(random_state=42)))]

run_classification(StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier()), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

## Voting

In [ ]:
from sklearn.ensemble import VotingClassifier


estimators = [('rf', RandomForestClassifier(criterion= 'entropy', n_estimators=100, random_state=42)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance')), ('bg', BaggingClassifier(n_estimators=100, random_state=42)), ('lsvc', LinearSVC(C=1, random_state=42))]

run_classification(VotingClassifier(estimators=estimators, voting='hard'), X_train_tfidf, X_test_tfidf, y_train1, y_test1)

## Deep Neural Networks

In [ ]:
# Load the augmented data from pickle file 
#with open('/content/Interim_data.pkl','rb') as f:
with open('Interim_data.pkl','rb') as f:
    clean_data_DL = pickle.load(f)

In [ ]:
clean_data_DL.isnull().sum()

In [ ]:
clean_data_DL['Final_Text'] = clean_data_DL['Final_Text'].replace(np.nan, '', regex=True)

In [ ]:
clean_data_DL.info()

In [ ]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
clean_data_DL['Assignment group LabelEncoded']= label_encoder.fit_transform(clean_data_DL['Assignment group']) 
  
clean_data_DL['Assignment group LabelEncoded'].unique()

In [ ]:
label_encoded_dict = dict(zip(clean_data_DL['Assignment group'].unique(), clean_data_DL['Assignment group LabelEncoded'].unique()))
len(label_encoded_dict)

In [ ]:
# Splitting Train Test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_data_DL['Final_Text'], clean_data_DL['Assignment group LabelEncoded'], test_size=0.3, random_state = 0, stratify=clean_data_DL['Assignment group LabelEncoded'])
print('\033[1mShape of the training set:\033[0m', X_train.shape, X_test.shape)
print('\033[1mShape of the test set:\033[0m', y_train.shape, y_test.shape)

### Create checkpoints function

In [ ]:
#Path where you want to save the weights, model and checkpoints
model_path = "Weights/"
%mkdir Weights

# Define model callbacks
def call_backs(name):
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=3)
    model_checkpoint =  ModelCheckpoint(model_path + name + '_epoch{epoch:02d}_loss{val_loss:.4f}.h5',
                                                               monitor='val_loss',
                                                               verbose=1,
                                                               save_best_only=True,
                                                               save_weights_only=False,
                                                               mode='min',
                                                               period=1)
    return [model_checkpoint, early_stopping]

In [ ]:
# Function to build Neural Network
def Build_Model_DNN_Text(shape, nClasses, dropout=0.3):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization

Tfidf_vect = TfidfVectorizer(max_features=2500)
Tfidf_vect.fit(clean_data_DL.Final_Text.astype(str))
X_train_tfidf = Tfidf_vect.transform(X_train)
X_test_tfidf = Tfidf_vect.transform(X_test)

# Instantiate the network
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 55)

In [ ]:
run_classification(model_DNN, X_train_tfidf, X_test_tfidf, y_train, y_test,pipelineRequired = False,isDeepModel=True, arch_name='DNN')

'''model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              callbacks=call_backs("NN"),
                              epochs=10,
                              batch_size=128,
                              verbose=2)
predicted = model_DNN.predict(X_test_tfidf)'''

### Extract Glove Embeddings

In [ ]:
#download the glove embedding zip file from http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
from zipfile import ZipFile
# Check if it is already extracted else Open the zipped file as readonly
if not os.path.isfile('glove.6B/glove.6B.200d.txt'):
    glove_embeddings = 'glove.6B.zip'
    #glove_embeddings = '/content/drive/MyDrive/Capstone/glove.6B.zip'
    with ZipFile(glove_embeddings, 'r') as archive:
        archive.extractall('glove.6B')

# List the files under extracted folder
os.listdir('glove.6B')

## Convolutional Neural Networks (CNN)

In [ ]:
gloveFileName = 'glove.6B/glove.6B.200d.txt'
#gloveFileName = '/content/glove.6B/glove.6B.200d.txt'
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM=200
MAX_NB_WORDS=75000

# Function to generate Embedding
def loadData_Tokenizer(X_train, X_test,filename):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(filename, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)


embedding_matrix = []

def buildEmbed_matrices(word_index,embedding_dim):
    embedding_matrix = np.random.random((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])), "into shape",str(len(embedding_vector)),
                      " Please make sure your"" EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
# Generate Glove embedded datasets
X_train_Glove, X_test_Glove, word_index, embeddings_index = loadData_Tokenizer(X_train,X_test,gloveFileName)
embedding_matrix = buildEmbed_matrices(word_index,EMBEDDING_DIM)

In [ ]:
def Build_Model_CNN_Text(word_index, embeddings_matrix, nclasses,dropout=0.5):
    """
        def buildModel_CNN(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
        word_index in word index ,
        embeddings_index is embeddings index, look at data_helper.py
        nClasses is number of classes,
        MAX_SEQUENCE_LENGTH is maximum lenght of text sequences,
        EMBEDDING_DIM is an int value for dimention of word embedding look at data_helper.py
    """
    model = Sequential()
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embeddings_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)
    # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    print("Filter  ",layer)
    for fl in range(0,layer):
        filter_sizes.append((fl+2))
    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    for fsz in filter_sizes:
        l_conv = Conv1D(node, kernel_size=fsz, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(5)(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)
    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv1D(node, 5, activation='relu')(l_merge)
    l_cov1 = Dropout(dropout)(l_cov1)
    l_batch1 = BatchNormalization()(l_cov1)
    l_pool1 = MaxPooling1D(5)(l_batch1)
    l_cov2 = Conv1D(node, 5, activation='relu')(l_pool1)
    l_cov2 = Dropout(dropout)(l_cov2)
    l_batch2 = BatchNormalization()(l_cov2)
    l_pool2 = MaxPooling1D(30)(l_batch2)
    l_flat = Flatten()(l_pool2)
    l_dense = Dense(1024, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)
    l_dense = Dense(512, activation='relu')(l_dense)
    l_dense = Dropout(dropout)(l_dense)
    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    print(model.summary())
    return model

In [ ]:
# Train the network and run classification
model_CNN = Build_Model_CNN_Text(word_index,embedding_matrix, 55)
run_classification(model_CNN, X_train_Glove, X_test_Glove, y_train, y_test,pipelineRequired = False,isDeepModel=True, arch_name='CNN')

## Recurrent Neural Networks (RNN) --> Gated Recurrent Unit (GRU)

In [ ]:
def Build_Model_RNN_Text(word_index, embeddings_matrix, nclasses,dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_matrix, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=100, dropout=0.5):
    word_index in word index ,
    embeddings_matrix is embeddings_matrix, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """
    model = Sequential()
    hidden_layer = 3
    gru_node = 32
    
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embeddings_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(GRU(gru_node, recurrent_dropout=0.2))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nclasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='sgd',
                      metrics=['accuracy'])
    
    print(model.summary())
    return model

In [ ]:
# Train the network and run classification
model_RNN = Build_Model_RNN_Text(word_index,embedding_matrix, 55)
run_classification(model_RNN, X_train_Glove, X_test_Glove, y_train, y_test,pipelineRequired = False,isDeepModel=True, arch_name='RNN')

## RNN with LSTM networks

In [ ]:
EMBEDDING_DIM = 200
gloveFileName = 'glove.6B/glove.6B.100d.txt'
#gloveFileName = '/content/glove.6B/glove.6B.200d.txt'

from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Activation
from keras.layers import Flatten, Permute, merge, Input
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input, Dense, multiply, concatenate, Dropout
from keras.layers import GRU, Bidirectional


def Build_Model_LTSM_Text(word_index, embeddings_matrix, nclasses):
    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256
    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embeddings_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Bidirectional(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(gru_node, recurrent_dropout=0.2)))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    print(model.summary())
    return model

In [ ]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test,gloveFileName)
embedding_matrix = buildEmbed_matrices(word_index,EMBEDDING_DIM)

model_LTSM = Build_Model_LTSM_Text(word_index,embedding_matrix, 55)
run_classification(model_LTSM, X_train_Glove, X_test_Glove, y_train, y_test,pipelineRequired = False,isDeepModel=True, arch_name='LSTM')